In [5]:
import cv2
import os

class StartDetector:
    def __init__(self, needle_img_name):
        script_dir = os.getcwd()
        self.needle_img_path = os.path.join(script_dir, 'Start_Testing', needle_img_name)
        self.needle_img = cv2.imread(self.needle_img_path, cv2.IMREAD_UNCHANGED)
        assert self.needle_img is not None, f"{self.needle_img_path} could not be read, check if it exists"

        if self.needle_img.shape[2] == 4:
            self.needle_img = self.needle_img[:, :, :3]

        self.needle_img_gray = cv2.cvtColor(self.needle_img, cv2.COLOR_BGR2GRAY)

    def detect(self, frame):
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        result = cv2.matchTemplate(frame_gray, self.needle_img_gray, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        return max_val, max_loc  # Returning the confidence value and the location of the best match

def find_matches_in_directory(directory_path, needle_img_name):
    detector = StartDetector(needle_img_name)
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for image_file in image_files:
        image_path = os.path.join(directory_path, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Could not read image {image_file}, skipping...")
            continue

        confidence, max_loc = detector.detect(image)
        threshold = 0.66  # Adjust threshold based on desired confidence level

        needle_height, needle_width = detector.needle_img_gray.shape[:2]

        if confidence >= threshold:
            print(f"Template found in {image_file} with confidence {confidence}")

            # Optional: Draw a rectangle around the detected region
            """ Uncomment to see the detected region
            top_left = max_loc
            bottom_right = (top_left[0] + needle_width, top_left[1] + needle_height)
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 3)

            result_image_path = os.path.join(directory_path, f"matched_{image_file}")
            cv2.imwrite(result_image_path, image)
            """
        else:
            print(f"Template not found in {image_file} with confidence {confidence}")

if __name__ == "__main__":
    test_directory = os.path.join(os.getcwd(), 'Start_Testing')
    needle_image = 'needle.png'
    find_matches_in_directory(test_directory, needle_image)


Template found in needle.png with confidence 1.0
Template found in Test1.png with confidence 0.6777594089508057
Template found in Test2.png with confidence 0.6911859512329102
Template found in Test3.png with confidence 0.6878242492675781
Template found in Test4.png with confidence 0.6877111196517944
Template found in Test5.png with confidence 0.6978583931922913
Template found in Test6.png with confidence 0.6918497681617737
Template found in Test7.png with confidence 0.7386835217475891
Template found in Test8.png with confidence 0.6749977469444275
Template not found in Test_Fail1.png with confidence 0.3888964354991913
Template not found in Test_Fail2.png with confidence 0.34733858704566956
Template not found in Test_Fail3.png with confidence 0.33909016847610474
Template not found in Test_Fail4.png with confidence 0.3736669421195984


In [7]:
import cv2
import os

class StartDetector:
    def __init__(self, needle_img_name):
        script_dir = os.getcwd()
        # Change to use 'Reset_Testing' directory
        self.needle_img_path = os.path.join(script_dir, 'Reset_Testing', needle_img_name)
        self.needle_img = cv2.imread(self.needle_img_path, cv2.IMREAD_UNCHANGED)
        assert self.needle_img is not None, f"{self.needle_img_path} could not be read, check if it exists"

        if self.needle_img.shape[2] == 4:
            self.needle_img = self.needle_img[:, :, :3]

        self.needle_img_gray = cv2.cvtColor(self.needle_img, cv2.COLOR_BGR2GRAY)

    def detect(self, frame):
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        result = cv2.matchTemplate(frame_gray, self.needle_img_gray, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        return max_val, max_loc  # Returning the confidence value and the location of the best match

def find_matches_in_directory(directory_path, needle_img_name):
    detector = StartDetector(needle_img_name)
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for image_file in image_files:
        image_path = os.path.join(directory_path, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Could not read image {image_file}, skipping...")
            continue

        confidence, max_loc = detector.detect(image)
        threshold = 0.66  # Adjust threshold based on desired confidence level

        needle_height, needle_width = detector.needle_img_gray.shape[:2]

        if confidence >= threshold:
            print(f"Template found in {image_file} with confidence {confidence}")

            # Optional: Draw a rectangle around the detected region
            """ Uncomment to see the detected region
            top_left = max_loc
            bottom_right = (top_left[0] + needle_width, top_left[1] + needle_height)
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 3)

            result_image_path = os.path.join(directory_path, f"matched_{image_file}")
            cv2.imwrite(result_image_path, image)
            """
        else:
            print(f"Template not found in {image_file} with confidence {confidence}")

if __name__ == "__main__":
    # Use 'Reset_Testing' directory for images to test
    test_directory = os.path.join(os.getcwd(), 'Reset_Testing')
    needle_image = 'needle.png'
    find_matches_in_directory(test_directory, needle_image)


Template found in needle.png with PSNR 361.20199909921956
Template not found in Test5.png with PSNR 9.62352774842059
Template not found in Test6.png with PSNR 9.564177771546783
Template not found in Test7.png with PSNR 9.764999028107821
Template not found in Test8.png with PSNR 9.401296869572926
Template not found in Test_1.png with PSNR 9.54643041731597
Template not found in Test_2.png with PSNR 9.582681617901661
Template not found in Test_3.png with PSNR 9.5518741781201
Template not found in Test_4.png with PSNR 9.654886420973925
Template not found in Test_Fail1.png with PSNR 9.827737089339513
Template not found in Test_Fail2.png with PSNR 6.133970156648006
Template not found in Test_Fail3.png with PSNR 7.910955445034748
Template not found in Test_Fail4.png with PSNR 9.452105084821422
